In [ ]:
!ls

drive  sample_data


In [ ]:
import pandas as pd
import numpy as np 
import re
import os

In [ ]:
path_input = "/content/drive/MyDrive/Resumes_Project/preprocess/dataset/annotation_english/words_output_dir/"
path_txt = "/content/drive/MyDrive/Resumes_Project/preprocess/dataset/annotation_english/txt_font/"
files = os.listdir(path_input)
len(files)

39

In [ ]:
r1 = re.compile(r'^[^@\s]+@[^@\s]+\.[^@\s]+$')
r2 = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
list_month = ["Jan", "January",
              "Feb", "February",
              "Mar", " March",
              "Apr", "April",
              "May", 
              "Jun", "June", 
              "Jul", "July",
              "Aug", "August",
              "Sep","Sept", "September",
              "Oct", "October",
              "Nov", "November",
              "Dec", "December"]
year = np.arange(2000, 2021)
list_year = []
for y in year:
  list_year.append(y)
  list_year.append(str(y))
list_year.append("Present")
list_year.append("Now")

keys_experience = ["experience", 
         "work experience",
         "work experiences",
         "professional experience",
         "professional working experience",
         "employment",
         "employment experience",
         "working experience",
         "working experiences",
         "employment history",
         "organisational experience"
         ]

keys_experience1 = ["experience", "emloyment", "work", "working", "professtional", "organisational"]
keys_experience2 = [ "work experience",
                    "work experiences",
                    "professional experience",
                    "employment experience",
                    "working experience",
                    "working experiences",
                    "employment history",
                    "organisational experience"
         ]
keys_experience3 = ["professional working experience"]
keys_skills = ["skills",
         "technical skills", 
         "skills and secondary school",
         "additional skills",
         "skills and activities",
         "skill",
         "professional skills",
         "special skills",
         "language proficiency and Skills",
         "computer research skills",
         "computer skills"
         ]
company_keys = ["ltd",
                "bhd"
                ]

keys_gpa = ["gpa",
            "cgpa",
            "spm"]         
keys_award = ["award",
              "awards",
              "educational awards & achievements",
              "academic awards"
              ]  
keys_edu = ["education", 
            "education and qualifications",
            "educations",
            "education & training",
            "education & academic qualifications",
            "education background",
            "academic credentials",
            "academic qualifications"
            ]  

keys_edu1 = ["education", "educations"]
keys_edu2 = ["education qualifications", "education qualification", "education training", "academic qualifications"
              "academic qualification", "education background",
             "academic credentials", "academic credential"]
keys_edu3 = ["education & training"]
keys_project = ["projects",
                "project experiences"]
keys_majors = ["bachelor", "major", "majors", "major:", "majors"]
keyword= ['name', 'name:', 'phone', 'phone:', 'address', 'address:', 'education', 'experience', 'experiences',
          'skill', 'skills', 'project', 'project', 'qualification', 'qualifications', 'academic', 'emloyment',
          "leadership", "communication"
          ] 

In [ ]:
def preprocess_file(file):
  df = pd.read_csv(path_input + file)
  # print(df['text'].head())
  df1 = df[['text', 'x0', 'y0', 'x1', 'y1','size','fontname', 'page_number']]

  # print(df1.shape)

  sizes = df[['size']].values.tolist()
  texts = df[['text']].values.tolist()
  sizes1 = []
  for i in range(len(sizes)):
    s = sizes[i]
    text1 = str(texts[i][0])
    if len(text1) == 1:
      if text1.isalpha() or text1.isdigit():
        sizes1.append(s[0])
    elif len(text1) >= 1:
      sizes1.append(s[0])

  sizes = set(sizes1)

  sizes = list(sizes)
  sizes.sort(reverse = True)
  # print(sizes)
  max_size1 = sizes[0]
  if len(sizes) > 1:
    max_size2 = sizes[1]
  else:
    max_size2 = 0
  data = np.array(df1)
  data = list(data)
  data1 = []
  for d in data:
    text = d[0]
    font = d[6]
    if  text != '' and not pd.isnull(text) and font != 'SymbolMT':
      data1.append(list(d))
  data = list(data1)

  # print(data)
  # print(len(data1))
  # print(len(data))
  # print("max_size1: ", max_size1)
  # print("max_size2: ", max_size2)

  return data, max_size1, max_size2

In [ ]:
def label_name(data, max_size1, max_size2):
  # print(data[: 10])
  # print(max_size1)
  # print(max_size2)
  flag = True 
  pivot = 0
  size_name = 0
  for i in range(0, min(20, len(data) - 1)):
    # print(i)
    # print(len(data))
    d =  data[i]
    text = d[0]
    xmin, ymin, xmax, ymax = d[1], d[2], d[3], d[4]
    size = d[5]
    font = d[6]
    if xmin < pivot:
      # print("L")
      break
    if size == max_size1 or size == max_size2:
      if text.isupper() or (text.istitle() and 'Bold' in font) or font=="BCDEEE+Arial-Black":
        # print("Lien")
        if text.lower() not in keyword:
          if size_name == 0:
            size_name = size
            data[i].append('name')
            flag = False
            pivot = xmin
          else:
            if size == size_name:
              data[i].append('name')
              flag = False
              pivot = xmin

          
  for i in range(len(data) - 30, len(data) - 5):
    d =  data[i]
    text = d[0]
    xmin, ymin, xmax, ymax = d[1], d[2], d[3], d[4]
    size = d[5]
    font = d[6]
    if xmin < pivot:
      # print("L")
      break
    if size == max_size1:
      if text.isupper() or (text.istitle() and 'Bold' in font) or font=="BCDEEE+Arial-Black":
        # print("Lien")
        if text.lower() not in keyword:
          if size_name == 0:
            size_name = size
            data[i].append('name')
            flag = False
            pivot = xmin
          else:
            if size == size_name:
              data[i].append('name')
              flag = False
              pivot = xmin

  

  if flag == True:
    for i in range(len(data) - 20, len(data) - 1):
      d =  data[i]
      text = d[0]
      xmin, ymin, xmax, ymax = d[1], d[2], d[3], d[4]
      size = d[5]
      font = d[6]
      if xmin < pivot:
        # print("L")
        break
      if size == max_size1 :

          # print("Lien")
        if text.lower() not in keyword:
          if size_name == 0:
            size_name = size
            data[i].append('name')
            # flag = False
            pivot = xmin
          else:
            if size == size_name:
              data[i].append('name')
              # flag = False
              pivot = xmin


  return data


In [ ]:
def label_email_phone(data):
  # flag_email = True 
  # flag_phone = True
  for i in range(0, len(data)-1):
    #check email
    text = data[i][0]
    # print(text)
    results1 = r1.findall(text)
    if len(results1) > 0 :
      data[i].append('email')
      # flag_email = False
      i = i + 1
      continue

    results2 = r2.findall(text)
    if len(results2) > 0 :
      data[i].append('phone')
      t = data[i-2][0]
      if 'phone' in t.lower() or 'tel' in t.lower():
        data[i-1].append('phone')
      # flag_phone = False
      i = i + 1
      continue

  return data

In [ ]:
def write_txt(file, data):
  print(data)
  
  num_pages = data[-1][7]
  # print("num_pages: ", num_pages)
  results = ""

  # for page_idx in range(1, num_pages + 1):
  page_prev = 1
  for i in range(0, len(data)):

    d = data[i]
    text = d[0]
    xmin= int(d[1])
    ymin=int(d[2])
    xmax=int(d[3])
    ymax=int(d[4])
    size = int(d[5])
    font = d[6]
    if "Bold" in font:
      font = 1
    else:
      font = 0


    page = int(d[7])
    if (len(d) >= 9):
      label = d[8]
    else:
      label = 'other'
    
    if page == page_prev:
      results += str(xmin) + "\t" + str(ymin) + "\t" + str(xmax) + "\t" + str(ymax) + "\t" + text + "\t" + str(size) + "\t" + str(font) + "\t" + label + "\n" 
    else:
      fout = path_txt + file + "_" + str(page_prev-1) + ".txt"
      fout = fout.replace("_chars.csv_words.csv", "")
      # print(fout)
      f = open(fout, "w")
      f.write(results[: -1])
      # print(results)
      f.close()
      results = ""
      results += str(xmin) + "\t" + str(ymin) + "\t" + str(xmax) + "\t" + str(ymax) + "\t" + text + "\t" + str(size) + "\t" + str(font) + "\t" + label + "\n"
      page_prev = page
  fout = path_txt + file + "_" + str(page_prev - 1) + ".txt"
  fout = fout.replace("_chars.csv_words.csv", "")
  # print(fout)
  f = open(fout, "w")
  f.write(results[: -1])
  f.close()

In [ ]:
def create_label(file):

  data,max_size1, max_size2 = preprocess_file(file)
  print("data2:", data)
  i = 0
  pivot = 0

  data = label_name(data, max_size1, max_size2)
  print("data2:", data)

  data = label_email_phone(data)

  len_data = len(data)
  i= 0
  while ( i < len_data - 1):
    d = data[i]
    data[i] = list(data[i])
    text = d[0]
    xmin, ymin, xmax, ymax = d[1], d[2], d[3], d[4]
    size = d[5]
    font = d[6]

    d_next = data[i+1]
    text_next = d_next[0]
    xmin_next, ymin_next, xmax_next, ymax_next = d_next[1], d_next[2], d_next[3], d_next[4]
    size_next = d_next[5]
    font_next = d_next[6]

    # check company
    company_size = 0

    school_size = 0
    school_font = ""

    if text.lower() in keys_experience1 and 'Bold' in font:
      # print("check company --------------------------------------------\n")
      if (i < len_data - 3) and (text.lower() + " " + text_next.lower() + data[i+2][0].lower()) in keys_experience3:
        pivot = xmin
        i = i+3
        text = data[i][0]
        if text in list_month or text in list_year:
          data[i].append("company_date")
          if data[i-2][-1] == "company_date":
            if len(data[i-1]) == 8:
              data[i-1].append("company_date")
            else:
              data[i-1][-1] = "company_date"
        elif text[:-1] in list_year:
          data[i].append("company_date")
        elif "/" in text or '-' in text or "Now" in text or "Present"in text:
          # TH 2/2017, 2016-Now
          for y in list_year:
            if str(y) in text:
              data[i].append("company_date")
        else:
          data[i].append("company")
          company_size = data[i][5]
     
      elif (i < len_data - 2) and (text.lower() + " " + text_next.lower()) in keys_experience2:
        pivot = xmin
        i = i + 2
        if text in list_month or text in list_year:
          data[i].append("company_date")
          if data[i-2][-1] == "company_date":
            if len(data[i-1]) == 8:
              data[i-1].append("company_date")
            else:
              data[i-1][-1] = "company_date"
        elif text[:-1] in list_year:
          data[i].append("company_date")
        elif "/" in text or '-' in text or "Now" in text or "Present"in text:
          # TH 2/2017, 2016-Now
          for y in list_year:
            if str(y) in text:
              data[i].append("company_date")
        else:
          data[i].append("company")
          company_size = data[i][5]
       
      else:
        pivot = xmin_next
        label = "other"
        label_next = "company"
        pivot = min(xmin, xmin_next)
        text = text_next
        i = i + 1
        if text in list_month or text in list_year:
          data[i].append("company_date")
          if data[i-2][-1] == "company_date":
            if len(data[i-1]) == 8:
              data[i-1].append("company_date")
            else:
              data[i-1][-1] = "company_date"
        elif text[:-1] in list_year:
          data[i].append("company_date")
        elif "/" in text or '-' in text or "Now" in text or "Present"in text:
          # TH 2/2017, 2016-Now
          for y in list_year:
            if str(y) in text:
              data[i].append("company_date")
        else:
          data[i].append("company")
          company_size = data[i][5]
        i = i + 1

      if i < len_data - 1:
        xmin = data[i+1][1]
      else:
        break

      while xmin >= pivot:
        if "Bold" in font:
          text = data[i][0]
          if text.lower() in list_month or text in list_year:
            data[i].append("company_date")
            if data[i-2][-1] == "company_date":
              if len(data[i-1]) == 8:
                data[i-1].append("company_date")
              else:
                data[i-1][-1] = "company_date"
          elif text[:-1] in list_year:
            data[i].append("company_date")
          elif "/" in text or '-' in text or "Now" in text or "Present"in text:
            # TH 2/2017, 2016-Now
            for y in list_year:
              if str(y) in text:
                data[i].append("company_date")
          else:
            flag = True
            for m in list_month:
              if m.lower() in text.lower():
                for y in list_year:
                  if str(y) in text:
                    data[i].append("company_date")
                    if data[i-2][-1] == "company_date":
                      if len(data[i-1]) == 8:
                        data[i-1].append("company_date")
                      else:
                        data[i-1][-1] = "company_date"
            if flag == True :
              if company_size == 0:
                data[i].append('company')
              elif data[i][5] == company_size:
                data[i].append('company')
            
            
        else:
          if text.lower() in list_month or text in list_year:
            data[i].append("company_date")
            if data[i-2][-1] == "company_date":
              if len(data[i-1]) == 8:
                data[i-1].append("company_date")
              else:
                data[i-1][-1] = "company_date"
              
          elif text[:-1] in list_year:
            data[i].append("company_date")
            if data[i-2][-1] == "company_date":
              if len(data[i-1]) == 8:
                data[i-1].append("company_date")
              else:
                data[i-1][-1] = "company_date"
          elif "/" in text or '-' in text or "Now" in text or "Present"in text:
            # TH 2/2017, 2016-Now
            for y in list_year:
              if str(y) in text:
                data[i].append("company_date")       
          else:
            for m in list_month:
              if m.lower() in text.lower():
                for y in list_year:
                  if str(y) in text:
                    data[i].append("company_date")
                    # flag = False
                    if data[i-2][-1] == "company_date":
                      if len(data[i-1]) == 8:
                        data[i-1].append("company_date")
                      else:
                        data[i-1][-1] = "company_date"
                      
          
        if i < len_data - 1:
          font = data[i+1][6]
          xmin = data[i+1][1]
          text = data[i+1][0]
          size = data[i+1][5]
          if xmin <= pivot and company_size != 0:
            if text.isupper() and size >= company_size and text.lower() in keys_edu1:
              i = i - 1
              break
            elif text.istitle() and "Bold" in font and text.lower() in keys_edu1:
              i = i - 1
              break
            else: 
              i = i + 1
          else:
            i = i + 1

        else: 
          break


    #check school
    
    elif text.lower() in keys_edu1 and 'Bold' in font:
      print("check school --------------------------------------------\n")
      if ( i < len_data - 3) and (text.lower() + " " + text_next.lower() + data[i+2][0].lower()) in keys_edu3:
        pivot = xmin
        i = i+3
        text = data[i][0]
        if text.lower() in list_month or text in list_year:
          data[i].append("school_date")
          if data[i-2][-1] == "school_date":
            if len(data[i-1]) == 8:
              data[i-1].append("school_date")
            else:
              data[i-1][-1] = "school_date"
        elif text[:-1] in list_year:
          data[i].append("school_date")
        elif "/" in text or '-' in text or "Now" in text or "Present"in text:
          # TH 2/2017, 2016-Now
          for y in list_year:
            if str(y) in text:
              data[i].append("school_date")
        else:
          data[i].append("school")
          school_size = data[i][5]
          school_font = data[i][6]
     
      elif (i < len_data -2) and  (text.lower() + " " + text_next.lower()) in keys_edu2:
        pivot = xmin
        i = i + 2
        text = data[i][0]
        if text.lower() in list_month or text in list_year:
          data[i].append("school_date")
          if data[i-2][-1] == "school_date":
            if len(data[i-1]) == 8:
              data[i-1].append("school_date")
            else:
              data[i-1][-1] = "school_date"
        elif text[:-1] in list_year:
          data[i].append("school_date")
        elif "/" in text or '-' in text or "Now" in text or "Present"in text:
          # TH 2/2017, 2016-Now
          for y in list_year:
            if str(y) in text:
              data[i].append("school_date")
        else:
          data[i].append("school")
          school_size = data[i][5]
          school_font = data[i][6]

      else:
        school_size = 0
        label = "other"
        label_next = "school"
        pivot = xmin
        if text_next.lower() in list_month or text_next in list_year:
          data[i+1].append("school_date")
          school_font = data[i+1][6]
          label_next = "school"
        elif text_next.lower() in keys_gpa:
          i = i + 1
          t = data[i][0]
          if len(t) != 1:
            data[i].append("grade")
            label_next = "school"
          else:
            i = i + 1
            t = data[i][0]
            if len(t) != 1:
              data[i].append("grade")
              label_next = "school"
   
        else:
          data[i+1].append(label_next)

        school_size = data[i+1][5]
        i = i + 1

        if i < len_data - 1:
          xmin = data[i+1][1]
        else:
          break
 
        while xmin >= pivot:
          # print("i: ", i)
          # if i == 378:
          #   break
          # print(xmin)
          # print(pivot)
          # print(text)
          # print(data[i][0])
          # print(data[i+1][0])
          text = data[i][0]

          # if i == 378:
          #   break
          if text.lower() in list_month or text in list_year:
            data[i].append("school_date")
            if data[i-2][-1] == "school_date":
              if len(data[i-1]) == 8:
                data[i-1].append("school_date")
              else:
                data[i-1][-1] = "school_date"
          elif text[ :-1] in list_year:
            data[i].append("school_date")
          elif "/" in text or '-' in text or "Now" in text or "Present"in text:
            # TH 2/2017, 2016-Now
            for y in list_year:
              if str(y) in text:
                data[i].append("school_date")
          elif text.lower() in keys_majors:
            label_next = "major"
            data[i].append("major")
          elif text.lower() in keys_gpa:
            data[i].append("grade")
            i = i + 1
            t = data[i][0]
            if len(t) != 1:
              data[i].append("grade")
              label_next = "school"
            else:
              i = i + 1
              t = data[i][0]
              if len(t) != 1:
                data[i].append("grade")
                label_next = "school"
          elif text.lower() in keyword:
            break
          else:
            flag = True 
            for m in list_month:
              if m.lower() in text.lower():
                for y in list_year:
                  if str(y) in text:
                    data[i].append("school_date")
                    label_next = "school"
                    flag = False
                    if data[i-2][-1] == "school_date":
                      if len(data[i-1]) == 8:
                        data[i-1].append("school_date")
                        label_next = "school"
                      else:
                        data[i-1][-1] = "school_date"
                        label_next = "school"

            if flag == True:
              if school_font == "":
                school_font = data[i][6]
                data[i].append(label_next)
          
              elif data[i][6] == school_font:
                data[i].append(label_next)

          if i < len_data - 1:
            # print("aaaaaaaaaaaaaa")
            font = data[i+1][6]
            xmin = data[i+1][1]
            text = data[i+1][0]
            # print("text: ", text)
            # print("school size: ", school_size)
            # print(data[i][-1])
            # print(data[i+1][5])
            
            if (text.isupper() and ("Bold" in font and text.istitle())):
              print("111111111111111")
              break

            if ( data[i+1][5] > school_size)  and data[i][-1] == 'school_date' :
                # print("lieen ddow")
                school_size = data[i+1][5]
            elif  (text.isupper() or ("Bold" in font and text.istitle())) and xmin <= pivot:
              # print(data)
              if ( data[i+1][5] > school_size) and data[i][-1] == 'school':
                # i = i - 1
                print("break")

                break
              elif text.lower() in keyword:
                # i = i - 1
                break
              else:
                i = i + 1
            else:
              # font = data[i+1][6]
              # xmin = data[i+1][1]
              # text = data[i+1][0]
              i = i + 1
          else:
            break

          # if i == 378:
          #   break

    # check project
    elif text.lower() in keys_project and 'Bold' in font:
      label_next = "project"
      pivot = xmin_next
      data[i+1].append(label_next)
      i = i + 1

      if i < len_data - 1:
        xmin = data[i+1][1]
      else:
        break
  
      while xmin >= pivot:
        if "Bold" in font:
          data[i].append(label_next)
          
        if i < len_data - 1:
          font = data[i+1][6]
          xmin = data[i+1][1]
          text = data[i+1][0]
          i = i + 1
        else:
          break

    else:
      i = i + 1

  print("data: ", data)
  write_txt(file, data)

In [ ]:
cnt = 0
for file in files:
  print(file)
  print(cnt)
  create_label(file)
  cnt += 1
  break


1_NET_Nguyễn Văn Hân.pdf_chars.csv_words.csv
0
data2: [['Applicant', 60.0, 672.075, 111.324, 684.075, 12.0, 'AAAISO+Roboto-Regular', 1], ['for:', 114.3, 672.075, 132.384, 684.075, 12.0, 'AAAISO+Roboto-Regular', 1], ['5', 60.0, 645.735, 71.46, 665.735, 20.0, 'AAAISP+Roboto-Bold', 1], ['Jr/Sr', 76.44, 645.735, 122.56, 665.735, 20.0, 'AAAISP+Roboto-Bold', 1], ['.NET', 127.54, 645.735, 170.4, 665.735, 20.0, 'AAAISP+Roboto-Bold', 1], ['Dev', 175.38, 645.735, 209.26, 665.735, 20.0, 'AAAISP+Roboto-Bold', 1], ['(C#,', 214.24, 645.735, 251.0, 665.735, 20.0, 'AAAISP+Roboto-Bold', 1], ['SQL', 255.98, 645.735, 293.46, 665.735, 20.0, 'AAAISP+Roboto-Bold', 1], ['server)', 298.44, 645.735, 361.06, 665.735, 20.0, 'AAAISP+Roboto-Bold', 1], ['VNPOST', 60.0, 618.346, 125.85799999999999, 635.346, 17.0, 'AAAISO+Roboto-Regular', 1], ['19', 60.0, 595.957, 79.108, 612.957, 17.0, 'AAAISO+Roboto-Regular', 1], ['August', 83.324, 595.957, 138.03, 612.957, 17.0, 'AAAISO+Roboto-Regular', 1], ['2020', 142.246, 5